In [21]:
using DataFrames
using XLSX
using Statistics

# Load the first sheet of the Excel file into a DataFrame
file_path = "/Users/marclambertes/Downloads/Database Men 2024-2025/Netherlands.xlsx"
df = DataFrame(XLSX.readtable(file_path, 1))  # Load the first sheet by index

# Define the columns to check
columns_to_check = ["Accurate back passes, %", "Accurate lateral passes, %", "Accurate short / medium passes, %"]

# Drop rows with missing values in the specified columns
df = filter(row -> all(!ismissing(row[col]) for col in columns_to_check), df)

# Filter out rows where any of the metrics are zero
df = filter(row -> all(row[col] != 0 for col in columns_to_check), df)

# Apply weights to the metrics
weights = Dict(
    "Accurate back passes, %" => 3,
    "Accurate lateral passes, %" => 2,
    "Accurate short / medium passes, %" => 1
)

# Calculate weighted values for each column
weighted_values = []
for col in collect(keys(weights))
    push!(weighted_values, df[!, col] .* weights[col])  # Multiply values by their weights
end
weighted_matrix = hcat(weighted_values...)

# Define the harmonic_mean function
function harmonic_mean(values::AbstractVector, weights::AbstractVector)
    println("Values: $values")
    println("Weights: $weights")
    return length(values) / sum(weights ./ values)
end

# Calculate the weighted harmonic mean
df[!, :Weighted_Harmonic_Mean] = [harmonic_mean(weighted_matrix[i, :], collect(values(weights))) for i in 1:size(df, 1)]

# Scale the weighted harmonic mean to a range of 0-100
min_whmean = minimum(df.Weighted_Harmonic_Mean)
max_whmean = maximum(df.Weighted_Harmonic_Mean)

df[!, :Weighted_Harmonic_Mean_Scaled] = 100 .* (df.Weighted_Harmonic_Mean .- min_whmean) ./ (max_whmean - min_whmean)

# Print player names along with their specific scaled weighted harmonic mean
for row in eachrow(df)
    println("Player: $(row.Player), Scaled Weighted Harmonic Mean: $(row.Weighted_Harmonic_Mean_Scaled)")
end

# Save the DataFrame to an Excel file
output_path = "/Users/marclambertes/Downloads/CPI_Bundesliga_Weighted.xlsx"
XLSX.openxlsx(output_path, mode="w") do xf
    XLSX.writetable(xf, "Sheet1", DataFrame(df))
end


Values: [199.14, 99.45, 300.0]
Weights: [2, 1, 3]


LoadError: MethodError: objects of type Int64 are not callable
The object of type `Int64` exists, but no method is defined for this combination of argument types when trying to treat it as a callable object.
Maybe you forgot to use an operator such as [36m*, ^, %, / etc. [39m?

In [8]:
import Pkg; Pkg.add("Statistics")

   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [10745b16] + Statistics v1.11.1
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
